In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from multiprocess import Pool
import update_idf_v1 as update_idf

In [3]:
import sys
sys.platform

'win32'

In [27]:
if __name__ ==  '__main__': 
    building_groups = ['pre1980', 'post1980', 'new']
    climate_zones = ['1a','2a','2b','3a','3b','3bc','3c','4a','4b','4c','5a','5b','6a','6b','7a','8a']
    run_updater = True
    all_result = []
    for bldg_group in building_groups:
        input_tuples = list(zip([bldg_group]*len(climate_zones),climate_zones, [run_updater]*len(climate_zones)))
        num_processors = len(climate_zones)
        with Pool(processes = num_processors) as p: 
            results = p.map_async(update_idf.run_cz_conversion, input_tuples)
            p.close()
            p.join()
            all_result.append(results)


In [25]:
all_result[2].get()

[[],
 [],
 [],
 [],
 ['C:\\Users\\Justin\\Documents\\GitHub\\cmip6_and_buildings\\eplus_simulations\\new\\idf_files\\v22_2\\3b_new_v22_2\\RefBldgFullServiceRestaurantNew2004_V22-2-0_3B_USA_NV_LAS_VEGAS.idf',
  'C:\\Users\\Justin\\Documents\\GitHub\\cmip6_and_buildings\\eplus_simulations\\new\\idf_files\\v22_2\\3b_new_v22_2\\RefBldgHospitalNew2004_V22-2-0_3B_USA_NV_LAS_VEGAS.idf',
  'C:\\Users\\Justin\\Documents\\GitHub\\cmip6_and_buildings\\eplus_simulations\\new\\idf_files\\v22_2\\3b_new_v22_2\\RefBldgLargeHotelNew2004_V22-2-0_3B_USA_NV_LAS_VEGAS.idf',
  'C:\\Users\\Justin\\Documents\\GitHub\\cmip6_and_buildings\\eplus_simulations\\new\\idf_files\\v22_2\\3b_new_v22_2\\RefBldgLargeOfficeNew2004_V22-2-0_3B_USA_NV_LAS_VEGAS.idf',
  'C:\\Users\\Justin\\Documents\\GitHub\\cmip6_and_buildings\\eplus_simulations\\new\\idf_files\\v22_2\\3b_new_v22_2\\RefBldgMediumOfficeNew2004_V22-2-0_3B_USA_NV_LAS_VEGAS.idf',
  'C:\\Users\\Justin\\Documents\\GitHub\\cmip6_and_buildings\\eplus_simulations\\ne